### Colab instructions

In [ ]:
!git clone https://github.com/GioC1810/aml23-ego.git
!pip install omegaconf coloredlogs wandb

### Compress a file

In [1]:
import lzma

with open('saved_features_action_net/new_emg_spectrogram_train.pkl', 'rb') as f_in:
    with lzma.open('saved_features_action_net/new_emg_spectrogram_train.pkl.xz', 'wb') as f_out:
        f_out.write(f_in.read())

## Epic kitchen

#### Feature extraction script

The script extract the features for 5, 10 and 25 num frames per clip

In [ ]:
%%bash

splits=("test" "train")
num_frames=(5 10 25)
dense_sampling=("True" "False")

for split in ${splits[@]}
do
  for frames in ${num_frames[@]}
  do
    for sampling in ${dense_sampling[@]}
    do
        python3 save_feat.py name=feature_extraction_D1 \
            config=configs/I3D_save_feat.yaml \
            dataset.shift=D1-D1 \
            save.num_frames_per_clip.RGB=$frames \
            save.dense_sampling.RGB=$sampling \
            dataset.RGB.data_path=../ek_data/frames 
    done
  done
done



#### Model training

Indicate shift, num frames per clip and model to use

In [ ]:
%%bash

python3 train_classifier_ek.py name=model_name \
  config=configs/default.yaml \
  dataset.shift=D1-D1 \
  dataset.train.num_frames_per_clip.RGB=5 \
  dataset.models.RGB=TRN_classifier

## Action net

#### Action net frame extraction

In [ ]:
%%bash

ffmpeg -i action-net/video/S04_video.mp4 -vf "fps=30,scale=456:256" -q:v 2 action-net/frames/S04_1/frame_%010d.jpg

#### Feature rgb extraction
Indicate num frames per clip

In [17]:
%%bash

python3 save_feat_action_net.py name=feature_rgb_extracted \
  config=configs/I3D_save_feat_an.yaml \
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=5 \
  split=train

2024-04-27 11:41:27 LOG INFO Feature Extraction
2024-04-27 11:41:27 LOG INFO Running with parameters: 
  action: save
  name: feature_rgb_extracted
  modality: ['RGB']
  total_batch: 128
  batch_size: 32
  gpus: None
  wandb_name: None
  resume_from: ./saved_models/I3D_SourceOnlyD1
  logname: save_D1-S04.log
  models_dir: models
  aggregation: True
  train:
    num_iter: 5000
    lr_steps: 3000
    eval_freq: 50
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 5
  test:
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 5
  dataset:
    annotations_path: action_net_annotations_training
    shift: D1-S04
    workers: 4
    stride: 2
    resolution: 224
    num_classes: 8
    RGB:
      data_path: action-net/frames
      tmpl: frame_{:010d}.jpg
      features_name: saved_features
    Event:
      rgb4e: 6
  models:
    RGB:
      model: I3D
      normalize: False
      kwargs:
      lr_steps: 3000
      lr: 0.0

In [7]:
%%bash

python3 save_feat_action_net.py
  config=configs/I3D_save_feat_an.yaml \
  split=test \
  dataset.train.num_frames_per_clip.RGB=25 \

2024-04-27 11:23:16 LOG INFO Feature Extraction
2024-04-27 11:23:16 LOG INFO Running with parameters: 
  action: train
  name: test
  modality: ['RGB']
  total_batch: 128
  batch_size: 32
  gpus: None
  wandb_name: None
  resume_from: None
  logname: None
  models_dir: models
  aggregation: True
  train:
    num_iter: 5000
    lr_steps: 3000
    eval_freq: 50
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 5
  test:
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 5
  dataset:
    annotations_path: train_val
    shift: D1-D1
    workers: 4
    stride: 2
    resolution: 224
    num_classes: 20
    RGB:
      data_path: saved_features
      tmpl: img_{:010d}.jpg
      features_name: saved_features
    Event:
      rgb4e: 6
  models:
    RGB:
      model: TRN_classifier
      normalize: False
      kwargs:
      lr_steps: 3000
      lr: 0.01
      sgd_momentum: 0.9
      weight_decay: 1e-07
  experiment_dir: 

CalledProcessError: Command 'b'\npython3 save_feat_action_net.py\n  config=configs/I3D_save_feat_an.yaml \\\n  split=test \\\n  dataset.train.num_frames_per_clip.RGB=25 \\\n'' returned non-zero exit status 127.

#### Decompress emg features train

In [18]:
import lzma

with lzma.open('saved_features_an_multimodal/features_emg_allData_train.pkl.xz', 'rb') as f_in:
    with open('saved_features_an_multimodal/features_emg_allData_train.pkl', 'wb') as f_out:
        f_out.write(f_in.read())
        

#### Split emg spectogram features in order to push it

In [9]:
import pickle

def split_dict(filename, output_prefix):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    
    length = len(data['features'])
    third_length = length // 3

    split = [data['features'][0:third_length], data['features'][third_length:2*third_length], data['features'][2*third_length:length]]

    for i in range(1,4):
        with open(output_prefix + f'_{i}.pkl', 'wb') as f:
            pickle.dump(split[i-1], f)

split_dict('saved_features_an_multimodal/features_emg_spectrogram_train.pkl', 'saved_features_an_multimodal/features_emg_spectrogram_train_split')

#### Decompress and join emg spectogram features

In [2]:
import pickle

def join_spectogram_feature(filename, output_name):
    joined_features = {'features': []}
    
    for i in range(3):
        with open(filename + f'_{i+1}.pkl', 'rb') as f:
            joined_features['features'].extend(pickle.load(f))
            
    
    with open(output_name, 'wb') as f:
        pickle.dump(joined_features, f)
        
join_spectogram_feature('saved_features_an_multimodal/features_emg_spectrogram_train_split', 
                        'saved_features_an_multimodal/features_emg_spectrogram_train.pkl')

#### Model training

In [4]:
%%bash

python3 train_classifier.py name=train_model_rgb \
  config=configs/default_an_rgb.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25 \
  dataset.models.RGB=TRN_classifier

#### Multimodal models


##### Late fusion model

###### Testing

In [ ]:
%%bash

python3 train_classifier_multimodal.py name=multimodal \
  config=configs/multimodal_an.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25 \
  models.RGB.model=Lstm_classifier \
  models.EMG.model=EMG_LSTM

##### Midl level fusion model

###### Mid level feature extraction

###### Training

In [ ]:
%%bash

python3 save_feat_midlevel_multimodal.py name=midlevel_feat \
  config=configs/mid_level_extraction.yaml \
  split=test
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=25 \
  models.RGB.model=TRN_classifier \
  models.EMG.model=EMG_LSTM

In [59]:
%%bash

python3 train_midlevel_classifier.py name=midlevel_classifier \
  config=configs/multimodal_midLevel_an.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=25

2024-04-28 19:24:37 LOG INFO Running with parameters: 
  action: train
  name: midllevel_classifier
  modality: ['FUSION']
  total_batch: 128
  batch_size: 32
  gpus: None
  wandb_name: None
  resume_from:
    FUSION: None
  logname: None
  models_dir: saved_models
  aggregation: True
  train:
    num_iter: 5000
    lr_steps: 3000
    eval_freq: 50
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 25
  test:
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 5
  dataset:
    annotations_path: an_multimodal_annotations
    shift: S04-S04
    workers: 4
    stride: 2
    resolution: 224
    num_classes: 20
    RGB:
      data_path: saved_features
      tmpl: img_{:010d}.jpg
      features_name: saved_features
    Event:
      rgb4e: 6
    FUSION:
      data_path: mid_level_features
      tmpl: img_{:010d}.jpg
      features_name: midlevel_feat
  models:
    RGB:
      model: TRN_classifier
      normalize: False